In [70]:
import pandas as pd
import re 
import spacy 
import os 
import numpy as np
import altair as alt
import textdistance
from sklearn.preprocessing import MultiLabelBinarizer
alt.data_transformers.enable('json')
pd.options.display.max_colwidth = 100
pd.options.display.max_seq_items = 2000

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
df = pd.read_csv("../../../Glentel Inc/HR Analytics - Documents/Capstone Data/ubc_mds_team_share/06_clean_data/05182020_cleaned_english_resumes_V1.0.csv")

In [4]:
df_train = pd.read_csv("../../../Glentel Inc/HR Analytics - Documents/Capstone Data/ubc_mds_team_share/02_eda_data/output/train_dataset.csv")

In [5]:
df_train_code = df_train[['employee_code', 'hp_class']]

In [6]:
df_resume = df[['employee_code', 'employee_name', 'clean_text', 'resume_text']]

In [7]:
df_resume_clean = df_train_code.merge(df_resume, on='employee_code')

In [8]:
df_resume_clean.head(2)

,employee_code,hp_class,employee_name,clean_text,resume_text
0,MZE,0.0,"Birla, Shelly",Avenue Tel reference available request objective obtain fulltime position enable utilize custome...,"SHELLY BIRLA 1293 Midland Avenue Toronto, ON M1K4H4 Tel: (647) 554 - 5181 shelly_birla@hotmail...."
1,MRU,0.0,"Williams, Dave",Resume obtain utilize opportunity grow benefit employer self Work Experience Assistant Store Man...,"Indeed Resume Dave Williams To obtain and utilize opportunities; while growing, benefiting the ..."


In [149]:
def prepocess(text): 
    labels = []
    
    text = str(text)
    text = text.lower()
    text = text.strip()
    text = re.sub('\W+',' ', text)
    text = re.sub("[\d-]", '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'''[\*\~]+''', "", text)
    
    # Replace a sequence of whitespaces by a single whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove other strange characters
    text = re.sub(r'''[\*\~]+''', "", text)
    
    #try spliting on education
    try:
        text = re.split('educati[^\s]+', text)[1]
        
        #hardplacement 
        if text == 'liberal arts': text = 'arts'
        if text == 'arts marketing': text = 'marketing'
        if text == 'arts psychology': text = 'psychology'
        if text == 'general arts': text = 'arts'
        text = re.sub("mba", 'business', text)
        text = re.sub("commerce", 'business', text)
        text = re.sub("hr", 'human resource', text)
        text = re.sub("mortgage", 'finance', text)
        text = re.sub("securities", 'finance', text)
        text = re.sub('computing', 'computer', text)
        text = re.sub('techonology', 'technology', text)
        text = re.sub('socialogy', 'sociology', text)
        text = re.sub('general arts', 'arts', text)
        text = re.sub('public', 'general', text)
        text = re.sub('o s s d', 'general', text)
        text = re.sub('informatics security', 'computer', text) 
        text = re.sub("bank", 'business', text)
        text = re.sub('informatics security', 'computer', text) 
        text = re.sub('softwar[^\s]+', 'computer', text) 
        text = re.sub('program[^\s]+', 'computer', text)
        
        #label search
        #accounting
        if re.search('accoun[^\s]+', text): labels.append("accounting_concentration")
        
        #arts
        #if arts and not digital arts
        if re.search('arts', text) and bool(re.search('digita[^\s]+', text)) == False: labels.append("arts_concentration")
        if re.search('arts', text) and bool(re.search('fine', text)) == False: labels.append("arts_concentration")
        if re.search('sociol[^\s]+', text): labels.append("arts_concentration")
        if re.search('psychol[^\s]+', text): labels.append("arts_concentration")
        if re.search('engli[^\s]+', text): labels.append("arts_concentration")
        if re.search('communicat[^\s]+', text): labels.append("arts_concentration")
        if re.search('criminol[^\s]+', text): labels.append("arts_concentration")            
            
        #business
        if re.search('busine[^\s]+', text): labels.append("business_concentration")
        if re.search('econom[^\s]+', text): labels.append("business_concentration")
        
        #computer system
        if re.search('comput[^\s]+', text): labels.append("computer_systems_concentration")
        
        #engineering
        if re.search('enginee[^\s]+', text): labels.append("engineering_concentration")
        
        #finance
        if re.search('financ[^\s]+', text): labels.append("finance_concentration")
        
        #human resource
        if re.search('human resour[^\s]+', text): labels.append("human_resource_concentration")
            
        #interactive arts and techology
        if re.search('media', text): labels.append('interactive_arts_and_technology_concentration')
        if re.search('technol[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
        if re.search('game', text): labels.append('interactive_arts_and_technology_concentration')
        if re.search('entertainm[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
        if re.search('informat[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
        if re.search('digit[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
        if re.search('Photogra[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
        if re.search('scree[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
        
        #marketing
        if re.search('marke[^\s]+', text): labels.append("marketing_concentration")
            
        #other
        if not labels:
            labels.append("other_concentration")
        
    except:
        #try to split on university
        try:
            text = re.split('univers[^\s]+', text)[1]
        
            #hardplacement 
            if text == 'liberal arts': text = 'arts'
            if text == 'arts marketing': text = 'marketing'
            if text == 'arts psychology': text = 'psychology'
            if text == 'general arts': text = 'arts'
            text = re.sub("mba", 'business', text)
            text = re.sub("commerce", 'business', text)
            text = re.sub("hr", 'human resource', text)
            text = re.sub("mortgage", 'finance', text)
            text = re.sub("securities", 'finance', text)
            text = re.sub('computing', 'computer', text)
            text = re.sub('techonology', 'technology', text)
            text = re.sub('socialogy', 'sociology', text)
            text = re.sub('general arts', 'arts', text)
            text = re.sub('public', 'general', text)
            text = re.sub('o s s d', 'general', text)
            text = re.sub('informatics security', 'computer', text) 
            text = re.sub("bank", 'business', text)
            text = re.sub('informatics security', 'computer', text) 
            text = re.sub('softwar[^\s]+', 'computer', text) 
            text = re.sub('program[^\s]+', 'computer', text)

            #label search
            #accounting
            if re.search('accoun[^\s]+', text): labels.append("accounting_concentration")

            #arts
            #if arts and not digital arts
            if re.search('arts', text) and bool(re.search('digita[^\s]+', text)) == False: labels.append("arts_concentration")
            if re.search('arts', text) and bool(re.search('fine', text)) == False: labels.append("arts_concentration")
            if re.search('sociol[^\s]+', text): labels.append("arts_concentration")
            if re.search('psychol[^\s]+', text): labels.append("arts_concentration")
            if re.search('engli[^\s]+', text): labels.append("arts_concentration")
            if re.search('communicat[^\s]+', text): labels.append("arts_concentration")
            if re.search('criminol[^\s]+', text): labels.append("arts_concentration")            

            #business
            if re.search('busine[^\s]+', text): labels.append("business_concentration")
            if re.search('econom[^\s]+', text): labels.append("business_concentration")

            #computer system
            if re.search('comput[^\s]+', text): labels.append("computer_systems_concentration")

            #engineering
            if re.search('enginee[^\s]+', text): labels.append("engineering_concentration")

            #finance
            if re.search('financ[^\s]+', text): labels.append("finance_concentration")

            #human resource
            if re.search('human resour[^\s]+', text): labels.append("human_resource_concentration")

            #interactive arts and techology
            if re.search('media', text): labels.append('interactive_arts_and_technology_concentration')
            if re.search('technol[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
            if re.search('game', text): labels.append('interactive_arts_and_technology_concentration')
            if re.search('entertainm[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
            if re.search('informat[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
            if re.search('digit[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
            if re.search('Photogra[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
            if re.search('scree[^\s]', text): labels.append('interactive_arts_and_technology_concentration')

            #marketing
            if re.search('marke[^\s]+', text): labels.append("marketing_concentration")

            #other
            if not labels:
                labels.append("no_education_section")
        
        except:
            #try spliting on college
            try:
                text = re.split('univers[^\s]+', text)[1]

                #hardplacement 
                if text == 'liberal arts': text = 'arts'
                if text == 'arts marketing': text = 'marketing'
                if text == 'arts psychology': text = 'psychology'
                if text == 'general arts': text = 'arts'
                text = re.sub("mba", 'business', text)
                text = re.sub("commerce", 'business', text)
                text = re.sub("hr", 'human resource', text)
                text = re.sub("mortgage", 'finance', text)
                text = re.sub("securities", 'finance', text)
                text = re.sub('computing', 'computer', text)
                text = re.sub('techonology', 'technology', text)
                text = re.sub('socialogy', 'sociology', text)
                text = re.sub('general arts', 'arts', text)
                text = re.sub('public', 'general', text)
                text = re.sub('o s s d', 'general', text)
                text = re.sub('informatics security', 'computer', text) 
                text = re.sub("bank", 'business', text)
                text = re.sub('informatics security', 'computer', text) 
                text = re.sub('softwar[^\s]+', 'computer', text) 
                text = re.sub('program[^\s]+', 'computer', text)

                #label search
                #accounting
                if re.search('accoun[^\s]+', text): labels.append("accounting_concentration")

                #arts
                #if arts and not digital arts
                if re.search('arts', text) and bool(re.search('digita[^\s]+', text)) == False: labels.append("arts_concentration")
                if re.search('arts', text) and bool(re.search('fine', text)) == False: labels.append("arts_concentration")
                if re.search('sociol[^\s]+', text): labels.append("arts_concentration")
                if re.search('psychol[^\s]+', text): labels.append("arts_concentration")
                if re.search('engli[^\s]+', text): labels.append("arts_concentration")
                if re.search('communicat[^\s]+', text): labels.append("arts_concentration")
                if re.search('criminol[^\s]+', text): labels.append("arts_concentration")            

                #business
                if re.search('busine[^\s]+', text): labels.append("business_concentration")
                if re.search('econom[^\s]+', text): labels.append("business_concentration")

                #computer system
                if re.search('comput[^\s]+', text): labels.append("computer_systems_concentration")

                #engineering
                if re.search('enginee[^\s]+', text): labels.append("engineering_concentration")

                #finance
                if re.search('financ[^\s]+', text): labels.append("finance_concentration")

                #human resource
                if re.search('human resour[^\s]+', text): labels.append("human_resource_concentration")

                #interactive arts and techology
                if re.search('media', text): labels.append('interactive_arts_and_technology_concentration')
                if re.search('technol[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
                if re.search('game', text): labels.append('interactive_arts_and_technology_concentration')
                if re.search('entertainm[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
                if re.search('informat[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
                if re.search('digit[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
                if re.search('Photogra[^\s]', text): labels.append('interactive_arts_and_technology_concentration')
                if re.search('scree[^\s]', text): labels.append('interactive_arts_and_technology_concentration')

                #marketing
                if re.search('marke[^\s]+', text): labels.append("marketing_concentration")

                #other
                if not labels:
                    labels.append("no_education_section")
            
            except:
                labels.append("no_education_section")

    return labels

In [152]:
df_resume_clean["educatation_concentration_list"] = df_resume_clean["clean_text"].apply(prepocess)

In [155]:
mlb = MultiLabelBinarizer()
df_resume_clean = df_resume_clean.join(pd.DataFrame(mlb.fit_transform(df_resume_clean.pop('educatation_concentration_list')),
                                   columns = mlb.classes_,
                                   index= df_resume_clean.index))

In [159]:
df_resume_clean = df_resume_clean.drop(['hp_class', 'employee_name', 'clean_text', 'resume_text'], axis=1)

In [164]:
df_resume_clean.head()

,employee_code,accounting_concentration,arts_concentration,business_concentration,computer_systems_concentration,engineering_concentration,finance_concentration,human_resource_concentration,interactive_arts_and_technology_concentration,marketing_concentration,no_education_section,other_concentration
0,MZE,0,1,0,0,0,0,0,0,0,0,0
1,MRU,0,0,1,0,1,0,0,1,0,0,0
2,LAP,1,0,1,1,0,0,1,1,0,0,0
3,MYD,0,0,0,0,0,0,0,0,0,1,0
4,JJB,0,0,1,0,0,0,0,1,0,0,0


In [162]:
df_resume_clean.describe()

,accounting_concentration,arts_concentration,business_concentration,computer_systems_concentration,engineering_concentration,finance_concentration,human_resource_concentration,interactive_arts_and_technology_concentration,marketing_concentration,no_education_section,other_concentration
count,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000
mean,0.097222,0.326389,0.298611,0.177083,0.065972,0.086806,0.086806,0.322917,0.145833,0.246528,0.142361
std,0.296776,0.469708,0.458446,0.382404,0.248666,0.282040,0.282040,0.468405,0.353553,0.431740,0.350028
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [165]:
df_resume_clean.to_csv('../../../Glentel Inc/HR Analytics - Documents/Capstone Data/ubc_mds_team_share/06_clean_data/auto_education_concentration.csv')